In [7]:
import numpy as np
from scipy.optimize import curve_fit
import pandas as pd

#### Резонансная частота

$\nu_0 = \dfrac{1}{2\pi\sqrt{LC}}$

In [58]:
def nu_0(L, C):
    return 1 / (2 * np.pi * np.sqrt(L * C))

In [59]:
nu_0(100e-3, 0.1e-6)

1591.5494309189535

## R = 0 Ом

freq = 1588 Hz

period = 35.322 ms

N = 30 cycles

#### Логарифмический декремент затухания для возрастающего участка

$\Theta = \dfrac1n \ln{\dfrac{U_0 - U_k}{U_0 - U_{k+n}}}$

In [60]:
def Theta_incr(U0, U1, U2, n):
    return 1/n * np.log((U0 - U1) / (U0 - U2))

In [61]:
df = pd.read_csv(r'raw-data/zug-incr_0-Ohm.csv')

In [62]:
U0 = df['U0[mV]']
U1 = df['U1[mV]']
U2 = df['U2[mV]']
n  = df['n']

In [63]:
Theta_incr(U0, U1, U2, n)

0    0.111672
1    0.115525
2    0.115721
3    0.113310
dtype: float64

#### Логарифмический декремент затухания для затухающего участка

$\Theta = \dfrac1n \ln{\dfrac{U_m}{U_{m+n}}}$

In [64]:
def Theta_decr(U1, U2, n):
    return 1/n * np.log((U1) / (U2))

In [65]:
df = pd.read_csv(r'raw-data/zug-decr_0-Ohm.csv')

In [66]:
U1 = df['U1[mV]']
U2 = df['U2[mV]']
n  = df['n']

In [67]:
Theta_decr(U1, U2, n)

0    0.096905
1    0.096698
2    0.091629
3    0.091964
dtype: float64

## R = 100 Ом

freq = 1588 Hz

period = 12 ms

N = 8 cycles

#### Логарифмический декремент затухания для возрастающего участка

In [68]:
df = pd.read_csv(r'raw-data/zug-incr_100-Ohm.csv')

In [69]:
U0 = df['U0[mV]']
U1 = df['U1[mV]']
U2 = df['U2[mV]']
n  = df['n']

In [70]:
Theta_incr(U0, U1, U2, n)

0    0.390079
1    0.433750
2    0.410715
3    0.449025
dtype: float64

#### Логарифмический декремент затухания для затухающего участка

In [71]:
df = pd.read_csv(r'raw-data/zug-decr_100-Ohm.csv')

In [72]:
U1 = df['U1[mV]']
U2 = df['U2[mV]']
n  = df['n']

In [73]:
Theta_decr(U1, U2, n)

0    0.302753
1    0.270310
2    0.282851
3    0.315313
dtype: float64

**50 Hz**

L = 100.10 mH

R1 = 0.028 Ohm

R2 = 100.02 Ohm

**500 Hz**

L = 100.08 mH

R1 = 0.029 Ohm

R2 = 100.02

**1500 Hz**

L = 100.10 mH

R1 = 0.03 Ohm

R2 = 100.02 Ohm